In [1]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.section import WD_SECTION
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from business_copy import TrafficReportGenerator
import warnings 
warnings.filterwarnings("ignore") 

In [2]:
report_generator = TrafficReportGenerator(file_path="raw data.xlsx")
df_sheet2 = pd.read_excel("raw data.xlsx", sheet_name="Sheet2")
if 'DATE TIME' not in df_sheet2.columns:
    raise KeyError("Column 'DATE TIME' not found in Sheet2")
df_sheet2['DATE TIME'] = pd.to_datetime(df_sheet2['DATE TIME'], errors='coerce')
if df_sheet2['DATE TIME'].notna().any():
    max_date = df_sheet2['DATE TIME'].max()
else:
    max_date = pd.Timestamp('2025-06-23 14:54:00') 
date_str = max_date.strftime('%B %d{0}, %Y').format(
    'th' if 10 <= max_date.day % 100 <= 20 else
    {1: 'st', 2: 'nd', 3: 'rd'}.get(max_date.day % 10, 'th')
).upper()
doc = Document()
def add_borderless_cover(doc, image_path):
    cover_section = doc.sections[0]
    cover_section.left_margin = Inches(0)
    cover_section.right_margin = Inches(0)
    cover_section.top_margin = Inches(0)
    cover_section.bottom_margin = Inches(0)
    cover_section.header_distance = Inches(0)
    cover_section.footer_distance = Inches(0)
    borderless_para = doc.add_paragraph()
    borderless_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
    try:
        borderless_run = borderless_para.add_run()
        borderless_run.add_picture(
            image_path,
            width=cover_section.page_width,
            height=cover_section.page_height
        )
        doc.add_section(WD_SECTION.NEW_PAGE)
        new_section = doc.sections[-1]
        new_section.left_margin = Inches(1)  # Reset to standard 1" margin
        new_section.right_margin = Inches(1)
        new_section.top_margin = Inches(1)
        new_section.bottom_margin = Inches(1)
        return True
    except Exception as e:
        print(f"Borderless cover failed: {str(e)}")
        return False
add_borderless_cover(doc, r'Z:\1. Reports\0. Templates\Script\cover_page.png')
doc.add_heading('TABLE OF CONTENTS', level=1)
toc_paragraphs = [
    ("1.0 General Summary", 2),("   1.1 Total Traffic Summary", 2),("   1.2 Overloads Summary", 3),
    ("   1.3 Missing Stations Report", 4),("2.0 Analysis of Total Traffic", 5),("   2.1 Total Traffic Per Station", 5),
    ("   2.2 Total Traffic per Vehicle Category", 6),("   2.3 Total Traffic for Trucks per Axle Configuration", 7),
    ("3.0 Unknowns Analysis", 8),("   Table 7: Unknowns Per Station Analysis", 8),
    ("4.0 Overloads Summary by Station", 9),("   4.1 Gross Vehicle Weight Overloads", 9),("   4.2 Axle Weight Overloads", 10),
    ("   4.3 Tagged Vehicles (GVW >2000kg)", 11),("   4.4 GVW >2000 Overloads per Axle Configuration", 12),("   4.5 Unknown Tagged Vehicles per Station", 13),]
for title, page in toc_paragraphs:
    p = doc.add_paragraph()
    run = p.add_run(title)
    run.add_text("\t" + "." * 60) 
    run.add_text(str(page)) 
    run.font.size = Pt(11)
    run.font.name = 'Arial'
    p.paragraph_format.left_indent = Inches(0.5 if title.startswith('   ') else 0)
    p.paragraph_format.space_after = Pt(6)
    tabs = p.paragraph_format.tab_stops
    tabs.clear_all()
    tabs.add_tab_stop(Inches(6.27), WD_ALIGN_PARAGRAPH.RIGHT)
section = doc.sections[-1] 
footer = section.footer
footer_text_paragraph = footer.add_paragraph()
footer_text_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
footer_text_run = footer_text_paragraph.add_run(
    f"KENHA/RD/MTCE/4192/2024          DAILY TRAFFIC ANALYSIS REPORT                 {date_str}"
)
footer_text_run.font.size = Pt(10)
footer_text_run.font.name = 'Arial'
footer_page_paragraph = footer.add_paragraph()
footer_page_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
run = footer_page_paragraph.add_run()
run.add_text("Page ")
fld = OxmlElement('w:fldSimple')
fld.set(qn('w:instr'), 'PAGE \\* MERGEFORMAT')
run._r.append(fld)
run.add_text(' of ')
fld_total = OxmlElement('w:fldSimple')
fld_total.set(qn('w:instr'), 'NUMPAGES \\* MERGEFORMAT')
run._r.append(fld_total)
run.font.size = Pt(10)
run.font.name = 'Arial'
doc.add_page_break() 
traffic_summary_df, traffic_summary_text = report_generator.generate_traffic_summary()
doc.add_heading('1.1 Traffic Summary', level=1)
doc.add_paragraph(traffic_summary_text)
doc.add_heading('Table 1: Traffic Summary', level=2)
table = doc.add_table(rows=len(traffic_summary_df) + 1, cols=len(traffic_summary_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(traffic_summary_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in traffic_summary_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
overloads_summary_df, overloads_summary_text = report_generator.generate_overloads_summary()
doc.add_heading('1.2 Overloads Summary', level=1)
doc.add_paragraph(overloads_summary_text)
doc.add_heading('Table 2: Overloads Summary', level=2)
table = doc.add_table(rows=len(overloads_summary_df) + 1, cols=len(overloads_summary_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(overloads_summary_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in overloads_summary_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
missing_stations_text = report_generator.generate_missing_stations_report()
doc.add_heading('1.3 Missing Stations Report', level=2)
doc.add_paragraph(missing_stations_text)
fig, station_summary_df, station_summary_text = report_generator.generate_station_traffic_analysis()
doc.add_heading('2.0 Analysis of Total Traffic', level=1)
doc.add_heading('2.1 Total Traffic Per Station', level=2)
doc.add_paragraph(station_summary_text)
doc.add_heading('Table 3: Station Traffic Summary', level=2)
table = doc.add_table(rows=len(station_summary_df) + 1, cols=len(station_summary_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(station_summary_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in station_summary_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
img_stream = io.BytesIO()
fig.savefig(img_stream, format='png', bbox_inches='tight')
doc.add_picture(img_stream, width=Inches(6))
plt.close(fig)
fig, vehicle_category_df, vehicle_category_text = report_generator.generate_vehicle_category_analysis()
doc.add_heading('2.2 Total Traffic per Vehicle Category', level=2)
doc.add_paragraph(vehicle_category_text)
doc.add_heading('Table 4: Vehicle Category Distribution', level=2)
table = doc.add_table(rows=len(vehicle_category_df) + 1, cols=len(vehicle_category_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(vehicle_category_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in vehicle_category_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
img_stream = io.BytesIO()
fig.savefig(img_stream, format='png', bbox_inches='tight')
doc.add_picture(img_stream, width=Inches(6))
plt.close(fig)
fig, trucks_distribution_df, trucks_distribution_text = report_generator.generate_trucks_distribution()
doc.add_heading('2.3 Total Traffic for Trucks per Axle Configuration', level=2)
doc.add_paragraph(trucks_distribution_text)
doc.add_heading('Table 5: Trucks Distribution', level=2)
table = doc.add_table(rows=len(trucks_distribution_df) + 1, cols=len(trucks_distribution_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(trucks_distribution_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in trucks_distribution_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
img_stream = io.BytesIO()
fig.savefig(img_stream, format='png', bbox_inches='tight')
doc.add_picture(img_stream, width=Inches(6))
plt.close(fig)
fig, unknowns_df, unknowns_text = report_generator.generate_unknowns_analysis()
doc.add_heading('3.0 Unknowns Analysis', level=1)
doc.add_paragraph(unknowns_text)
doc.add_heading('Table 6: Unknowns Per Station Analysis', level=2)
table = doc.add_table(rows=len(unknowns_df) + 1, cols=len(unknowns_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(unknowns_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in unknowns_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
img_stream = io.BytesIO()
fig.savefig(img_stream, format='png', bbox_inches='tight')
doc.add_picture(img_stream, width=Inches(6))
plt.close(fig)
figures, overloads_summary_station_df, overloads_summary_station_text = report_generator.generate_overloads_summary_by_station()
doc.add_heading('4.0 Overloads Summary by Station', level=1)
doc.add_heading('Table 7: Overloads Summary by Station', level=2)
table = doc.add_table(rows=len(overloads_summary_station_df) + 1, cols=len(overloads_summary_station_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(overloads_summary_station_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in overloads_summary_station_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
for i, (text, fig) in enumerate(zip(overloads_summary_station_text, figures)):
    if i == 0:
        doc.add_heading('4.1 Gross Vehicle Weight Overloads', level=2)
    elif i == 1:
        doc.add_heading('4.2 Axle Weight Overloads', level=2)
    elif i == 2:
        doc.add_heading('4.3 Tagged Vehicles (GVW >2000kg)', level=2)
    
    doc.add_paragraph(text)
    img_stream = io.BytesIO()
    fig.savefig(img_stream, format='png', bbox_inches='tight')
    doc.add_picture(img_stream, width=Inches(6))
    plt.close(fig)
fig, axle_config_df, axle_config_text = report_generator.generate_axle_config_summary()
doc.add_heading('4.4 GVW >2000 Overloads per Axle Configuration', level=2)
doc.add_paragraph(axle_config_text)
doc.add_heading('Table 8: Axle Configuration Summary', level=2)
table = doc.add_table(rows=len(axle_config_df) + 1, cols=len(axle_config_df.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(axle_config_df.columns):
    table.rows[0].cells[i].text = col_name
for i, row in axle_config_df.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
img_stream = io.BytesIO()
fig.savefig(img_stream, format='png', bbox_inches='tight')
doc.add_picture(img_stream, width=Inches(6))
plt.close(fig)
fig, final_result, sign_off_table, unknown_tags_text = report_generator.generate_unknown_tags_analysis()
doc.add_heading('4.5 Unknown Tagged Vehicles per Station', level=2)
doc.add_paragraph(unknown_tags_text)
doc.add_heading('Table 9: Unknown Tags Analysis', level=2)
table = doc.add_table(rows=len(final_result) + 1, cols=len(final_result.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(final_result.columns):
    table.rows[0].cells[i].text = col_name
for i, row in final_result.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)
img_stream = io.BytesIO()
fig.savefig(img_stream, format='png', bbox_inches='tight')
doc.add_picture(img_stream, width=Inches(6))
plt.close(fig)
doc.add_heading('Sign-Off', level=2)
table = doc.add_table(rows=len(sign_off_table) + 1, cols=len(sign_off_table.columns))
table.style = 'Table Grid'
for i, col_name in enumerate(sign_off_table.columns):
    table.rows[0].cells[i].text = col_name
for i, row in sign_off_table.iterrows():
    for j, value in enumerate(row):
        table.rows[i + 1].cells[j].text = str(value)

In [3]:
# Save the document with dynamic file name
output_filename = f"Virtual Stations Daily Traffic Analysis Report For {date_str}.docx"
doc.save(output_filename)